<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/projet_adversarial_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projet "apprentissage par ordinateur et exemples adversaires"

Le projet porte sur un problème d'apprentissage "simple" (classification binaire), mais, à travers le prisme des exemples adversaires.

Précisément, le projet s'effectue sur les données http://www.nlpr.ia.ac.cn/pal/trafficdata/recognition.html

- Ce jeu de données permet de faire de la classification multiclasses, mais, il sera rendu binaire pour plus de simplicité: les 16 premières classes seront regroupés (limitation de vitesse et interdiction de tourner) dans la classe 1, tous les autres paneaux sont regroupés dans la classe 2.

- De plus, afin de faciliter le traitement des images, toutes les images seront ramener à des images 96x96.

**Mais** l'objectif n'est pas seulement d'apprendre un classifier (binaire) sur ces images 96x96. L'objectif est 

- de générer des images adversaires pour tromper un tel classifier

**ou**

- d'apprendre un classifier à la fois "robuste" et performant


##prise en main des données

Télécharger les données. Le code ci dessous (**CHEMINS VERS LES DONNEES À MODIFIER**) permet de charger les données i.e. transformer cet ensemble d'image en une grosse matrice (4 matrices préciséments Xtrain,Xtest,Ytrain,Ytest).

In [0]:
import csv
import numpy as np
import PIL
import PIL.Image

print("LOADING DATA")
print("rescaling all images at 96x96")
print("merging classes in 2 super classes (speed limit and turn interdiction vs all)")
Xtrain = np.zeros((4170,3,96,96),dtype=int)
Ytrain = np.zeros(4170,dtype=int)
Xtest = np.zeros((1994,3,96,96),dtype=int)
Ytest = np.zeros(1994,dtype=int)


with open("build/TsignRecgTrain4170Annotation.txt") as f:
    reader = csv.reader(f,delimiter=";")
    
    for i,row in enumerate(reader):
        image3D = np.asarray(PIL.Image.open("build/tsrd-train/"+row[0]).convert("RGB").copy().resize((96,96)),dtype=float)
        Xtrain[i] = np.transpose(image3D,(2,0,1))
        if int(row[7])<16:
            Ytrain[i] = 0
        else:
            Ytrain[i] = 1
        
with open("build/TsignRecgTest1994Annotation.txt") as f:
    reader = csv.reader(f,delimiter=";")
    
    for i,row in enumerate(reader):
        image3D = np.asarray(PIL.Image.open("build/TSRD-Test/"+row[0]).convert("RGB").copy().resize((96,96)),dtype=float)
        Xtest[i] = np.transpose(image3D,(2,0,1))
        if int(row[7])<16:
            Ytest[i] = 0
        else:
            Ytest[i] = 1
        
print("nb of samples with class 0/1 in train=",4170-np.sum(Ytrain),"/",np.sum(Ytrain))
print("nb of samples with class 0/1 in test=",4170-np.sum(Ytest),"/",np.sum(Ytest))

##référence sur les adversariaux

Indépendamment, voilà quelques liens vers des articles de référence sur les adversariaux

* Une introduction au problème des adversariaux : www.tensorflow.org/tutorials/generative/adversarial_fgsm
* L'article correspondant arxiv.org/abs/1412.6572

Du coté de la méthode de référence de "protection" contre les adversariaux
* apprendre sur des données adversaires limites leur effet arxiv.org/abs/1710.10571
* apprendre sur la pire données adversaires possibles permet d'avoir une robustesse certaines arxiv.org/abs/1711.00851

**Important** : générer des données adversaires est très simple quand on a accès au gradient vis à vis de l'image elle même (typiquement si le modèle est du deep learning). Mais c'est possible aussi en approximant le gradient d'un modèle non deep : arxiv.org/pdf/1602.02697.pdf !

##La dérivée par rapport aux données

Nous avons vu en cours que la façon même d'apprendre les poids des réseaux de neurones passe 
- par le calcul des dérivées par rapport aux valeurs des neurones
- sachant que la dérivée par rapport à un neurones se calcule via la dérivée par rapport aux neurones suivants

Cela s'applique aux valeurs des entrées !

Informatiquement, il suffit de préciser qu'on veut stocker ces dérivées pour qu'elle soit calculées en utilisant *loss.backward()*


In [0]:
import torch
import torch.nn as nn

import torch.autograd
import torch.autograd.variable

mytarget = torch.randn(1,1,4,4)
myinput = torch.autograd.Variable(torch.randn(1,3,4,4),requires_grad=True)
mymodel = nn.Conv2d(3,1,kernel_size=3, padding=1)

myloss = torch.sum((mytarget-mymodel(myinput))*(mytarget-mymodel(myinput)))

print("classical learning aims to optimize weights of mymodel to decrease loss")
print("but here, one could optimize value of myinput to increase the loss eventually producing an adversarial example")

print(myloss)
myloss.backward()

print("here is the gradient of myloss regarding myinput",myinput.grad.data)


##Une baseline

Comme baseline pour commencer, il est possible (mais pas obligatoire) de considérer l'encodeur d'un VGG16 (2 convolutions - 1 pooling - 2 convolutions - 1 pooling - 3 convolutions - 1 pooling - 3 convolution - 1 pooling - 3 convolution - 1 pooling).

Appliqué à une image 3x96x96, la sortie sera une image 512x3x3 à laquel on appliquera un dernier pooling 3x3 stride 1 sans padding pour obtenir un vecteur de dimension 512, il faudra ensuite ajouter un dernier neurone pour passer de 512 à 2 valeurs (pour apprendre avec une cross entropy - 1 valeur si vous voulez apprendre en en hinge loss).

**le modèle VGG est directement téléchargeable depuis des dépôts pytorch**

In [0]:
import torchvision

monvgg16 = torchvision.models.vgg16(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
tensortest = torch.randn(1,3,96,96)
print(monvgg16.features(tensortest).shape)

torch.Size([1, 512, 3, 3])


On a bien l'encodeur qu'on voulait

In [0]:
monvgg16.avgpool = nn.AvgPool2d(kernel_size=(3, 3))
monvgg16.classifier = None
monvgg16.classifier = nn.Linear(512,2)
print(monvgg16(tensortest).shape)

torch.Size([1, 2])


en remplaçant le classifier, on a obtient un modèle adapté à classer des images 96x96 !

Il ne reste plus qu'à l'entrainer sur cette base...

##Maintenant à vous de jouer !